# Train a K-Means Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

K-Means is a basic but powerful clustering method which is optimized via Expectation Maximization. It randomly selects K data points in X, and computes which samples are close to these points. For every cluster of points, a mean is computed, and this becomes the new centroid.

cuML's KMeans supports the scalable KMeans++ intialization method. This method is more stable than randomnly selecting K points.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input

1. [Setup K-Means using cuML](#rbm-model)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#gpu)<br>

1. [Setup K-Means using sklearning](#rbm-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#cpu)<br>

<a id = "rbm-model"></a>
## Step 1 : Setup KMeans model using cuML

### Prepare directory and file for writing KMeans engine.

In [1]:
from pathlib import Path
model_dir = f'/data/models' 
model_main = f'KMeans_main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


### create a KMeans Model based on cuML on GPU

In [2]:
%%writefile {model_dir}/{model_main}


import cudf
import cupy
import matplotlib.pyplot as plt
from cuml.cluster import KMeans as cuKMeans
from cuml.datasets import make_blobs
from sklearn.cluster import KMeans as skKMeans
from sklearn.metrics import adjusted_rand_score
import os
import datetime

# %matplotlib inline

# specify the cache location to /gpfy since ~/.cache is not available
os.environ["CUPY_CACHE_DIR"]="/gpfs/mydatafs/models/"

# Define Parameters for a large regression
n_samples = 100000
n_features = 2

n_clusters = 5
random_state = 0

# Generate Data
device_data, device_labels = make_blobs(n_samples=n_samples,
                                        n_features=n_features,
                                        centers=n_clusters,
                                        random_state=random_state,
                                        cluster_std=0.1)

device_data = cudf.DataFrame(device_data)
device_labels = cudf.Series(device_labels)

#  # Copy dataset from GPU memory to host memory.
#  # This is done to later compare CPU and GPU results.
host_data = device_data.to_pandas()
host_labels = device_labels.to_pandas()

# kmeans_sk = skKMeans(init="k-means++",
#                      n_clusters=n_clusters,
#                      n_jobs=-1,
#                     random_state=random_state)

# kmeans_sk.fit(host_data)

# cuML Model
kmeans_cuml = cuKMeans(init="k-means||",
                       n_clusters=n_clusters,
                       oversampling_factor=40,
                       random_state=random_state)

# Fit
start = datetime.datetime.now()
kmeans_cuml.fit(device_data)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Visualize 
fig = plt.figure(figsize=(16, 10))
plt.scatter(host_data.iloc[:, 0], host_data.iloc[:, 1], c=host_labels, s=50, cmap='viridis')

# #plot the sklearn kmeans centers with blue filled circles
# centers_sk = kmeans_sk.cluster_centers_
# plt.scatter(centers_sk[:,0], centers_sk[:,1], c='blue', s=100, alpha=.5)

#plot the cuml kmeans centers with red circle outlines
centers_cuml = kmeans_cuml.cluster_centers_
plt.scatter(cupy.asnumpy(centers_cuml[0].values), 
            cupy.asnumpy(centers_cuml[1].values), 
            facecolors = 'none', edgecolors='red', s=100)

plt.title('cuml and sklearn kmeans clustering')

# plt.show()
plot_file = "./kmeans_gpu.png"
plt.savefig(plot_file)

# Evaluate
start = datetime.datetime.now()
cuml_score = adjusted_rand_score(host_labels, kmeans_cuml.labels_.to_array())
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

print("score (cuML): %s" % cuml_score)

Overwriting /data/models/KMeans_main.py


## Step 2 :  Training the model on GPU with Watson Machine Learning Accelerator

<a id = "gpu"></a>
#### Prepare the model lib for running on GPU:

In [3]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


#### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [4]:
hostname='wmla-console-wmla.apps.wml1x180.ma.platformlab.ibm.com'  # please enter Watson Machine Learning Accelerator host name
# login='username:password' # please enter the login and password
login='admin:p7PMrMMknVQzEb3ptyj0D6XRTO5PQjYL'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)

a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)

https://wmla-console-wmla.apps.wml1x180.ma.platformlab.ibm.com/auth/v1/logon


In [5]:
dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

In [6]:
# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)


In [7]:
#### define the status checking fuction

In [8]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

In [9]:
model_file = model_dir+"/"+model_main
files = {'file': open(model_file , 'rb')}

args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --conda-env-name rapids-21.06  \
                     --model-main /gpfs/mydatafs/models/KMeans_main.py --workerDeviceType gpu'
                    # --epochs 5 --batch-size 10000 --workerDeviceType gpu'
                    # --model-main '+  "/gpfs/mydatafs/" + model_file + ' --epochs 5 --batch-size 10000'

In [10]:
starttime = datetime.datetime.now()

r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
if not r.ok:
    print('submit job failed: code=%s, %s'%(r.status_code, r.content))
        
job_status = query_job_status(r.json(),refresh_rate=5)

endtime = datetime.datetime.now()

print("\nTraining cost: ", (endtime - starttime).seconds, " seconds.")

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-195,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-195,admin,FINISHED,wmla-195,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-195/_submitted_code,SingleNodeTensorflowTrain,2021-07-02T05:53:47Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-195',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --conda-env-name '
          'rapids-21.06                       --model-main '
          '/gpfs/mydatafs/models/KMeans_main.py --workerDeviceType gpu ',
  'createTime': '2021-07-02T05:53:47Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-195',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-195',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-195/_submitted_code'}

Training cost:  612  seconds.


## Step 3 :  Setup KMeans model using SK-Learning
### create a KMeans Model based on sk-learning on CPU

In [25]:
%%writefile {model_dir}/{model_main}

#import cudf
#import cupy
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans as skKMeans
from sklearn.metrics import adjusted_rand_score
import os
import datetime

# %matplotlib inline

# specify the cache location to /gpfy since ~/.cache is not available
os.environ["CUPY_CACHE_DIR"]="/gpfs/mydatafs/models/"

# Define Parameters for a large regression
n_samples = 100000
n_features = 2

n_clusters = 5
random_state = 0

# Generate Data
host_data, host_labels = make_blobs(n_samples=n_samples,
                                        n_features=n_features,
                                        centers=n_clusters,
                                        random_state=random_state,
                                        cluster_std=0.1)

kmeans_sk = skKMeans(init="k-means++",
                     n_clusters=n_clusters,
                     n_jobs=-1,
                    random_state=random_state)

kmeans_sk.fit(host_data)

# Fit
start = datetime.datetime.now()
kmeans_sk.fit(host_data)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Visualize 
fig = plt.figure(figsize=(16, 10))
plt.scatter(host_data[:, 0], host_data[:, 1], c=host_labels, s=50, cmap='viridis')

#plot the sklearn kmeans centers with blue filled circles
centers_sk = kmeans_sk.cluster_centers_
plt.scatter(centers_sk[:,0], centers_sk[:,1], c='blue', s=100, alpha=.5)



plt.title('sklearn kmeans clustering')

# plt.show()
plot_file = "./kmeans_cpu.png"
plt.savefig(plot_file)

# Evaluate
start = datetime.datetime.now()
sk_score = adjusted_rand_score(host_labels, kmeans_sk.labels_)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

print("score (sklearning): %s" % sk_score)

Overwriting /data/models/KMeans_main.py


## Step 4 :  Training the SK-Learning model on CPU with Watson Machine Learning Accelerator

In [26]:
model_file = model_dir+"/"+model_main
files = {'file': open(model_file , 'rb')}

args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --conda-env-name rapids-21.06  \
                     --model-main /gpfs/mydatafs/models/KMeans_main.py --workerDeviceType cpu'
                    # --epochs 5 --batch-size 10000 --workerDeviceType gpu'
                    # --model-main '+  "/gpfs/mydatafs/" + model_file + ' --epochs 5 --batch-size 10000'

In [27]:
import datetime

starttime = datetime.datetime.now()

# ! python {model_dir}/{model_main} # --no-cuda --epochs 5 --batch-size 10000
r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
if not r.ok:
    print('submit job failed: code=%s, %s'%(r.status_code, r.content))
        
job_status = query_job_status(r.json(),refresh_rate=5)

endtime = datetime.datetime.now()
print("Training cost: ", (endtime - starttime).seconds, " seconds.")


Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-201,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-201,admin,FINISHED,wmla-201,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-201/_submitted_code,SingleNodeTensorflowTrain,2021-07-02T06:25:45Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-201',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --conda-env-name '
          'rapids-21.06                       --model-main '
          '/gpfs/mydatafs/models/KMeans_main.py --workerDeviceType cpu ',
  'createTime': '2021-07-02T06:25:45Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-201',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-201',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-201/_submitted_code'}
Training cost:  37  seconds.
